In [190]:
import pandas as pd
import pprint
import ast
import json
import numpy as np
from scipy.stats import entropy
from scipy.special import softmax
import numpy as np
import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy
import ast



In [191]:
from dict import news_labels, yahoo_labels, newsgroups_labels, depression_labels, emotion_labels, rating_labels, depression, newsgroups, go_emotions, ag_news, yahoo, sst5

In [192]:
model = 'llama3'
dataset_tag = 'sst5'
# dict_map = yahoo_labels
dict_num = sst5
random = 24
N = dict_num['num_classes'] * dict_num['num_samples']
print(N)
selected_samples = f'random{random}_{N}'
selected_method = f'entropy_from_random{random}_{N}'

60


In [193]:
dict_num, dataset_tag

({'num_classes': 5, 'num_samples': 12, 'num_samples_smaller': 6}, 'sst5')

In [194]:

prompt = 'random_samples'
temp = 0
exp = 1

In [195]:
selcted_samples = ''

In [196]:
path = f'../RANDOM/results/{prompt}_temp{0}/{model}/{dataset_tag}_{model}_{prompt}_{model}_{selected_samples}_temp{temp}_exp{exp}.csv'

In [197]:
import os
output_path = f'selected_samples/{dataset_tag}/{model}_{selected_method}.csv'
folder = os.path.dirname(output_path)
os.makedirs(folder, exist_ok=True)

In [198]:

def calculate_metric_with_true_label(df, column_name):
    

    def parse_top_logprobs(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"❌ Nie udało się sparsować: {val[:60]}... ➡ {e}")
                return None
        return val

    df['top_logprobs'] = df['top_logprobs'].apply(parse_top_logprobs)
    
    df['logprobs_cal'] = df['top_logprobs'].apply(
    lambda x: np.array([item['logprob'] for item in x if isinstance(item, dict) and 'logprob' in item])
    if isinstance(x, list) else np.array([])
)



    # Przekształcenie logprobs do prawdopodobieństw przez softmax
    df['probs'] = df['logprobs_cal'].apply(
        lambda logprobs: softmax(logprobs) if len(logprobs) > 0 else np.array([])
    )

    # Entropia rozkładu
    df['entropy'] = df['probs'].apply(
        lambda probs: entropy(probs, base=2) if len(probs) > 0 else np.nan
    )

    # Margin: różnica między top-1 a top-2
    df['margin'] = df['probs'].apply(
        lambda probs: np.sort(probs)[-1] - np.sort(probs)[-2] if len(probs) > 1 else np.nan
    )

    # KL divergence względem idealnego rozkładu (1.0 na poprawnej etykiecie)
    def kl_div(probs, top_logprobs, label):
        if not isinstance(top_logprobs, list) or len(probs) == 0:
            return np.nan

        try:
            # lista tokenów jako stringi
            tokens = [str(item['token']).strip().lower() for item in top_logprobs if isinstance(item, dict) and 'token' in item]
            label_str = str(label).strip().lower()

            if label_str not in tokens:
                return np.nan

            match_idx = tokens.index(label_str)
            ideal_dist = np.zeros_like(probs)
            ideal_dist[match_idx] = 1.0

            return entropy(ideal_dist, probs, base=2)

        except Exception as e:
            print(f"⚠️ Błąd w kl_div: {e}")
            return np.nan


    # df['kl_divergence'] = df.apply(
    #     lambda row: kl_div(row['probs'], row['top_logprobs'], row[column_name]), axis=1
    # )
    
    df['kl_divergence'] = df['probs'].apply(
    lambda probs: np.sum(probs * np.log(probs / (1 / len(probs)))) if len(probs) > 0 else np.nan
        )
    
    # df['top_prob'] = df['probs'].apply(lambda p: np.max(p) if len(p) > 0 else np.nan)
    # df['kl_divergence'] = 2 * (1 - df['top_prob'])



    # Normalizacja (min-max)
    for col in ['entropy', 'margin', 'kl_divergence']:
        norm_col = f'normalized_{col}'
        min_val = df[col].min()
        max_val = df[col].max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            df[norm_col] = 0.0

    return df




In [199]:


df = pd.read_csv(path)


In [200]:
df['original_label_map'] = df['original_label']

In [201]:
df

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,original_label_map
0,Though the opera itself takes place mostly ind...,2,"{'content': [{'token': '2', 'logprob': -0.0245...","[{'token': '2', 'logprob': -0.0245339404791593...",3,False,3
1,If you can read the subtitles ( the opera is s...,3,"{'content': [{'token': '3', 'logprob': -0.1483...","[{'token': '3', 'logprob': -0.1483518332242965...",4,False,4
2,( `` Safe Conduct '' ) is a long movie at 163 ...,4,"{'content': [{'token': '4', 'logprob': -0.0279...","[{'token': '4', 'logprob': -0.0279490798711776...",4,False,4
3,The beauty of Alexander Payne 's ode to the Ev...,5,"{'content': [{'token': '5', 'logprob': -0.3935...","[{'token': '5', 'logprob': -0.3935191929340362...",4,False,4
4,The fourth in a series that I 'll bet most par...,2,"{'content': [{'token': '2', 'logprob': -0.6790...","[{'token': '2', 'logprob': -0.679002046585083}...",3,False,3
...,...,...,...,...,...,...,...
995,"In other words , it 's badder than bad .",1,"{'content': [{'token': '1', 'logprob': -0.0100...","[{'token': '1', 'logprob': -0.0100219622254371...",1,False,1
996,The direction occasionally rises to the level ...,2,"{'content': [{'token': '2', 'logprob': -0.4790...","[{'token': '2', 'logprob': -0.4790137410163879...",2,False,2
997,The movie is ingenious fun .,5,"{'content': [{'token': '5', 'logprob': -0.5281...","[{'token': '5', 'logprob': -0.5281342267990112...",5,False,5
998,The cast comes through even when the movie doe...,3,"{'content': [{'token': '3', 'logprob': -0.1459...","[{'token': '3', 'logprob': -0.1459727883338928...",3,False,3


In [202]:
# import re
# import numpy as np

# def extract_number(text):
#     match = re.search(r'\b\d+\b', str(text))
#     if match:
#         return int(match.group())
#     else:
#         return np.nan  # lub inna wartość domyślna

# df['output'] = df['output'].apply(extract_number)
# df['original_label_map'] = df['original_label_map'].apply(extract_number)

# df['output'] = df['output'].astype('int64')
# df['original_label_map'] = df['original_label_map'].astype('int64')



In [203]:
print(df['original_label_map'].isnull().sum())

0


In [204]:
df_metric = calculate_metric_with_true_label(df, 'original_label_map')
df_metric

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,original_label_map,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
0,Though the opera itself takes place mostly ind...,2,"{'content': [{'token': '2', 'logprob': -0.0245...","[{'token': '2', 'logprob': -0.0245339404791593...",3,False,3,"[-0.024533940479159355, -4.024533748626709, -5...","[0.9758120149597772, 0.01787262391819772, 0.00...",0.193241,0.957939,2.861788,0.068154,0.958065,0.931846
1,If you can read the subtitles ( the opera is s...,3,"{'content': [{'token': '3', 'logprob': -0.1483...","[{'token': '3', 'logprob': -0.1483518332242965...",4,False,4,"[-0.14835183322429657, -2.0233519077301025, -5...","[0.8622027509299113, 0.13222306443107631, 0.00...",0.620708,0.729980,2.565490,0.220517,0.730075,0.779483
2,( `` Safe Conduct '' ) is a long movie at 163 ...,4,"{'content': [{'token': '4', 'logprob': -0.0279...","[{'token': '4', 'logprob': -0.0279490798711776...",4,False,4,"[-0.027949079871177673, -3.902949094772339, -5...","[0.9724948951669604, 0.020183487333985905, 0.0...",0.214127,0.952311,2.847311,0.075598,0.952436,0.924402
3,The beauty of Alexander Payne 's ode to the Ev...,5,"{'content': [{'token': '5', 'logprob': -0.3935...","[{'token': '5', 'logprob': -0.3935191929340362...",4,False,4,"[-0.39351919293403625, -1.1435191631317139, -5...","[0.6747366584857324, 0.31872303887558334, 0.00...",0.961287,0.356014,2.329419,0.341910,0.356060,0.658090
4,The fourth in a series that I 'll bet most par...,2,"{'content': [{'token': '2', 'logprob': -0.6790...","[{'token': '2', 'logprob': -0.679002046585083}...",3,False,3,"[-0.679002046585083, -1.179002046585083, -2.17...","[0.5081290338404804, 0.3081958381144096, 0.113...",1.750969,0.199933,1.782053,0.623379,0.199959,0.376621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"In other words , it 's badder than bad .",1,"{'content': [{'token': '1', 'logprob': -0.0100...","[{'token': '1', 'logprob': -0.0100219622254371...",1,False,1,"[-0.010021962225437164, -4.760022163391113, -6...","[0.9900467289629203, 0.008565580812732744, 0.0...",0.087764,0.981481,2.934899,0.030558,0.981610,0.969442
996,The direction occasionally rises to the level ...,2,"{'content': [{'token': '2', 'logprob': -0.4790...","[{'token': '2', 'logprob': -0.4790137410163879...",2,False,2,"[-0.47901374101638794, -0.9790137410163879, -5...","[0.6194296060985849, 0.3757030476325114, 0.003...",1.002257,0.243727,2.301021,0.356513,0.243758,0.643487
997,The movie is ingenious fun .,5,"{'content': [{'token': '5', 'logprob': -0.5281...","[{'token': '5', 'logprob': -0.5281342267990112...",5,False,5,"[-0.5281342267990112, -0.9031342267990112, -5....","[0.5897411286046019, 0.4053227549520309, 0.004...",1.018351,0.184418,2.289865,0.362250,0.184443,0.637750
998,The cast comes through even when the movie doe...,3,"{'content': [{'token': '3', 'logprob': -0.1459...","[{'token': '3', 'logprob': -0.1459727883338928...",3,False,3,"[-0.14597278833389282, -2.145972728729248, -4....","[0.8643031813992177, 0.11697072282896673, 0.01...",0.666375,0.747332,2.533836,0.236794,0.747430,0.763206


In [205]:
print(type(df['original_label_map'].iloc[0]))
print(type(df['output'].iloc[0]))


<class 'numpy.int64'>
<class 'numpy.int64'>


In [206]:
(df['original_label_map'] != df['output']).sum()

387

In [207]:
N

60

In [208]:
(df.sort_values(by='entropy', ascending=False)['original_label_map'].head(N) != df.sort_values(by='entropy', ascending=False)['output'].head(N)).sum()

34

In [209]:
top_50_uncertain = df.sort_values(by='entropy', ascending=False).head(N)
top_50_uncertain

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,original_label_map,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
930,than ` Magnifique ' .,3,"{'content': [{'token': '3', 'logprob': -1.2714...","[{'token': '3', 'logprob': -1.2714394330978394...",3,False,3,"[-1.2714394330978394, -1.2714394330978394, -2....","[0.2858223627744326, 0.2858223627744326, 0.119...",2.807609,0.000000,1.049646,1.000000,0.000000,0.000000
208,Only at the prospect of Beck 's next project .,3,"{'content': [{'token': '3', 'logprob': -1.1703...","[{'token': '3', 'logprob': -1.1703182458877563...",3,False,3,"[-1.1703182458877563, -1.4203182458877563, -1....","[0.3115694769409359, 0.2426505526227489, 0.188...",2.441241,0.068919,1.303593,0.869414,0.068928,0.130586
92,It was a dark and stormy night ...,3,"{'content': [{'token': '3', 'logprob': -0.6773...","[{'token': '3', 'logprob': -0.677360475063324}...",3,False,3,"[-0.677360475063324, -2.0523605346679688, -2.3...","[0.5137565853336632, 0.1298979996352658, 0.101...",2.436006,0.383859,1.307222,0.867548,0.383909,0.132452
625,The question hanging over The Time Machine is ...,2,"{'content': [{'token': '2', 'logprob': -1.0278...","[{'token': '2', 'logprob': -1.0278832912445068...",3,False,3,"[-1.0278832912445068, -1.0278832912445068, -2....","[0.3604151614049789, 0.3604151614049789, 0.070...",2.359430,0.000000,1.360300,0.840254,0.000000,0.159746
689,Every so often a film comes along that is so i...,1,"{'content': [{'token': '1', 'logprob': -1.2331...","[{'token': '1', 'logprob': -1.2331125736236572...",2,False,2,"[-1.2331125736236572, -1.4831125736236572, -1....","[0.292058209464644, 0.22745516223349715, 0.200...",2.359105,0.064603,1.360525,0.840138,0.064612,0.159862
539,"Boy oh boy , it 's a howler .",2,"{'content': [{'token': '2', 'logprob': -1.1057...","[{'token': '2', 'logprob': -1.1057617664337158...",3,False,3,"[-1.1057617664337158, -1.4807617664337158, -1....","[0.3317506831391792, 0.22800868775313884, 0.17...",2.270470,0.103742,1.421962,0.808546,0.103756,0.191454
44,It 's not the least of Afghan tragedies that t...,3,"{'content': [{'token': '3', 'logprob': -1.1216...","[{'token': '3', 'logprob': -1.1216611862182617...",2,False,2,"[-1.1216611862182617, -1.1216611862182617, -1....","[0.32636551620528875, 0.32636551620528875, 0.1...",2.211631,0.000000,1.462746,0.787574,0.000000,0.212426
468,"But when it does , you 're entirely unprepared .",3,"{'content': [{'token': '3', 'logprob': -0.8116...","[{'token': '3', 'logprob': -0.8116999864578247...",3,False,3,"[-0.8116999864578247, -1.3116999864578247, -2....","[0.445690571300399, 0.2703249962385315, 0.1126...",2.163914,0.175366,1.495822,0.770566,0.175389,0.229434
479,Bella is the picture of health with boundless ...,3,"{'content': [{'token': '3', 'logprob': -0.5268...","[{'token': '3', 'logprob': -0.5268280506134033...",3,False,3,"[-0.5268280506134033, -2.2768280506134033, -2....","[0.5946201529696876, 0.10332948883664954, 0.10...",2.150390,0.491291,1.505196,0.765745,0.491355,0.234255
303,You want to call Domino 's .,2,"{'content': [{'token': '2', 'logprob': -1.1711...","[{'token': '2', 'logprob': -1.171165943145752}...",3,False,3,"[-1.171165943145752, -1.171165943145752, -1.29...","[0.31082031140593835, 0.31082031140593835, 0.2...",2.051035,0.000000,1.574063,0.730332,0.000000,0.269668


In [210]:
top_50_uncertain = df.sort_values(by='entropy', ascending=False).head(N)[['text', 'original_label']].rename(columns={'original_label': 'label'})


In [211]:
top_50_uncertain

,text,label
930,than ` Magnifique ' .,3
208,Only at the prospect of Beck 's next project .,3
92,It was a dark and stormy night ...,3
625,The question hanging over The Time Machine is ...,3
689,Every so often a film comes along that is so i...,2
539,"Boy oh boy , it 's a howler .",3
44,It 's not the least of Afghan tragedies that t...,2
468,"But when it does , you 're entirely unprepared .",3
479,Bella is the picture of health with boundless ...,3
303,You want to call Domino 's .,3


In [212]:
top_50_uncertain.to_csv(output_path, index=False)